## Create Database in R
This notebook creates the hexagon database using R's `sf` package. 

The workflow:
* Import the `sf` package.
* Import the needs and hexagon shapefiles as spatial feature dataframes 
 * Transform the hexagon feature class to the same coord. system as the needs
* Convert the hexagon geometries to a series of centroid geometries
* Iterate through each feature in the needs spatial dataframe
 * Create a new column in the hexagon dataframe corresponding to the need feature
  * Set default values to '0'
 * Extract the geometry for the feature
 * Create a boolean mask of all centroids that intersect the need geometry
 * Using the mask, set all corresponding features in the new column to '1'
 * Move to the next feature in the needs dataframe

In [ ]:
#Import packages
library(sf)
library(dplyr)

### Read in and prep the spatial data

In [ ]:
#set input & output filenames
fname_need <- '../Data/raw/Basic_Data/1_Conservation_Need.shp'
fname_hex <- '../Data/raw/Hexagon_files/100km2_D1_LamAzimNeed.shp'
fname_output <- '../scratch/Hexagons_from_R.shp'

In [ ]:
#Read in the needs shapefile
fcNeed <- st_read(fname_need,quiet=TRUE)

In [ ]:
#Read in the hexagons shapefile
fcHex <- st_read(fname_hex,quiet=TRUE)

In [ ]:
#Transform the hexagons feature class to match the coordinate system of the needs
fcHex <- st_transform(fcHex,st_crs(fcNeed))

In [ ]:
#Create a geoseries of centroids from the hexagon polygon geoseries
hex.centroids <- st_centroid(fcHex['geometry'])

## Construct the hexagon database

In [ ]:
#Iterate through each record in the needs feature class
counter <- 1
for (row in 1:nrow(fcNeed)) {
    #Set the output column name
    col.name <- paste("need",counter,sep='')
    #Increment the counter
    counter <- counter + 1
    #Get the shape
    sel.shape <- fcHex[row,'geometry']
    #Construct a Boolean mask where the hexagon features intersect the shape
    sel.mask = st_intersects(sel.shape,hex.centroids,sparse=FALSE)
    #Set default value in a new column to zero
    fcHex[col.name] = 0
    #Update masked features to a value of 1
    fcHex[sel.mask,col.name] = 1  
}

In [ ]:
#Create a total column
fcHex$NeedTotal <- as.data.frame(fcHex) %>% 
 select(starts_with("need")) %>%
 rowSums()

In [ ]:
colnames(fcHex)

In [ ]:
#Write results
st_write(fcHex,fname_output,delete_layer=TRUE)